# 2. Ocean part

## 2.1 Read and process data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore

In [2]:
# Read the raw CSV file.
mackay_raw = pd.read_csv("Mackay.csv")  # Pioneer area data
mackay_raw.head() # Cheek the columns information

In [3]:
m = mackay_raw.copy() # Make a copy for next use
# Parse the date column     
m["Date"] = pd.to_datetime(m["Aggregated Date/Time"], errors="coerce")
# Simplify column names 
m = m.rename(columns={"Site Name": "Site", "Longitude": "Lon", "Latitude": "Lat","Variable":"Var"})
# Process data 1: Replace string 'no data' with NaN 
for c in ["Depth","Lat","Lon","mean","median","p5","p95"]:
    if c in m.columns:
        m.loc[m[c].astype(str).str.lower().eq("no data"), c] = np.nan
# Process data 2: Converts valid numbers to floats
for c in ["Depth","Lat","Lon","mean","median","p5","p95"]:
    if c in m.columns:
        m[c] = pd.to_numeric(m[c], errors="coerce")
# Remove any bogus depth flags
m = m[(m["Depth"] > -1000) & (m["Depth"] < 1000)]

In [4]:
# Select the aimed depth and time period
m8 = m[(m["Depth"] == -8.8) & (m["Date"] >= "2014-01-01") & (m["Date"] <= "2019-04-30")].copy()
# Keep columns that are useful for this topic
keep_cols = ["Date","Site","Lat","Lon","Depth","Var","mean","median","p5","p95"]
m8 = m8[keep_cols].sort_values(["Site","Var","Date"]).reset_index(drop=True)
# Check the prosessed data
m8.head()

In [5]:
# Transforms the table from long format to wide format for easy use
mackay_wide_8m = (m8
    .pivot_table(index=["Date","Site","Depth"], columns="Var", values="mean", aggfunc="mean")
    .reset_index()
    .sort_values(["Site","Date"]))
# Check the wide format
mackay_wide_8m.head()

In [6]:
#Save processed data for easy access next time
m8.to_csv("mackay_8m_2014_2019_long.csv", index=False) #Mackey -8.8m, long format data from 2014 to 2019
mackay_wide_8m.to_csv("mackay_8m_2014_2019_wide.csv", index=False) #Mackey -8.8m, wide format data from 2014 to 2019

## 2.2 Time series and lag analysis of Omega vs. Nutrition and Omega vs. Dust for 4 sites

In [7]:
# Read the processed wide CSV file.
mackay = pd.read_csv("mackay_8m_2014_2019_wide.csv") 

In [8]:
# Processed data
DEPTH = -8.8
mackay = mackay.loc[mackay["Depth"] == DEPTH, ["Date", "Site", "Depth",
                                               "omega_ar", "DIN", "DIP", "Dust"]].copy()
mackay["Date"] = pd.to_datetime(mackay["Date"])
mackay = mackay.sort_values(["Site", "Date"]).reset_index(drop=True)

# Standardize to reduce space error
for col in ["omega_ar", "DIN", "DIP", "Dust"]:
    mackay[col + "_z"] = (
        mackay.groupby("Site")[col]
              .transform(lambda s: (s - s.mean()) / s.std()))
# Combine DIN and DIP into one nutrient index 
mackay["nutrient_z"] = mackay[["DIN_z", "DIP_z"]].mean(axis=1)
# 3-month centred smoothing to reduce short-term noise
mackay["omega_z_roll"]   = (mackay.groupby("Site")["omega_ar_z"]
                                 .apply(lambda s: s.rolling(3, center=True, min_periods=1).mean())
                                 .reset_index(level=0, drop=True))
mackay["nutrient_z_roll"] = (mackay.groupby("Site")["nutrient_z"]
                                  .apply(lambda s: s.rolling(3, center=True, min_periods=1).mean())
                                  .reset_index(level=0, drop=True))
mackay["dust_z_roll"]     = (mackay.groupby("Site")["Dust_z"]
                                  .apply(lambda s: s.rolling(3, center=True, min_periods=1).mean())
                                  .reset_index(level=0, drop=True))
# Select meaningful site 
sites = ["site 2", "site 3", "site 4", "site 5"] #Site 1 doesn't has data at -8.8m

In [9]:
# Set the same y-limits 
ymin = mackay[["omega_z_roll", "nutrient_z_roll"]].min().min()
ymax = mackay[["omega_z_roll", "nutrient_z_roll"]].max().max()
# Plot the figure
fig, axes = plt.subplots(len(sites), 1, figsize=(10, 12), sharex=True)
for ax, site in zip(axes, sites):
    site_data = (mackay[mackay["Site"] == site]
                 .sort_values("Date"))

    # Plot Ωarag (black) vs Nutrient index (red) — both 3-month means
    ax.plot(site_data["Date"], site_data["omega_z_roll"],
            color="black", label="Ωarag (z)")
    ax.plot(site_data["Date"], site_data["nutrient_z_roll"],
            color="red",   label="Nutrient index (DIN+DIP, z)")
    ax.axhline(0, color="gray", lw=0.7)
    ax.set_ylabel(site)
    ax.set_ylim(ymin, ymax)
    # Show legend
    if site == sites[0]:
        ax.legend(frameon=False, ncols=2, loc="upper right")
axes[-1].set_xlabel("Year")
fig.suptitle("Pioneer (Mackay) — Ωarag vs Nutrients (Z-scores, −8.8 m, 3-month mean)",
             y=1.02)
plt.tight_layout()
plt.show()
# Save the figure 
fig.savefig("Ocean_1_mackay_omega_vs_nutrient.png", dpi=600, bbox_inches="tight")

In [10]:
# Define a function for lag correlation
def prep_site_for_lag(df, site, depth=-8.8):
    s = (df[(df["Site"] == site) & (df["Depth"] == depth)]
           .sort_values("Date")
           .loc[:, ["Date", "omega_z_roll", "nutrient_z_roll"]]
           .set_index("Date"))
    return s.dropna()
def lag_corr(series_x, series_y, lags=range(0, 7)):    
    out = []
    for k in lags:
        r = series_x.corr(series_y.shift(k))
        out.append(r)
    return pd.Series(out, index=list(lags))
LAGS = range(0, 7)

In [11]:
fig, axes = plt.subplots(nrows=len(sites), ncols=1, figsize=(10, 10), sharex=True)
for ax, site in zip(axes, sites):
    s = prep_site_for_lag(mackay, site)
    r = lag_corr(s["omega_z_roll"], s["nutrient_z_roll"], LAGS)
    ax.plot(r.index, r.values, marker="o", lw=1.8, color="black")
    ax.axhline(0, color="gray", lw=0.7)
    ax.set_ylabel(site)
    min_lag = r.idxmin()
    min_val = r.min()
    ax.scatter(min_lag, min_val, color="red", zorder=5)
    ax.text(min_lag, min_val + 0.08, f"{min_val:.2f}",
            color="red", ha="center", va="bottom", fontsize=18, fontweight="bold")
axes[-1].set_xlabel("Lag (months) — nutrients lead Ωarag")
fig.suptitle("Pioneer (Mackay) — Lag analysis between Ωarag and Nutrients (−8.8 m, 3-month mean)",
             y=1.02)
plt.tight_layout()
plt.show()
# Save the figure 
fig.savefig("Ocean_2_lag_mackay_omega_vs_nutrient.png", dpi=600, bbox_inches="tight")

In [13]:
# Same plot for dust: omage vs dust
ymin = mackay[["omega_z_roll", "dust_z_roll"]].min().min()
ymax = mackay[["omega_z_roll", "dust_z_roll"]].max().max()
fig, axes = plt.subplots(len(sites), 1, figsize=(10, 12), sharex=True)
for ax, site in zip(axes, sites):
    sd = (mackay[mackay["Site"] == site]
          .sort_values("Date"))
    ax.plot(sd["Date"], sd["omega_z_roll"], color="black", label="Ωarag")
    ax.plot(sd["Date"], sd["dust_z_roll"],  color="red", label="Dust")
    ax.axhline(0, color="gray", lw=0.7)
    ax.set_ylabel(site)
    ax.set_ylim(ymin, ymax)
    if site == sites[0]:
        ax.legend(frameon=False, ncols=2)
axes[-1].set_xlabel("Year")
fig.suptitle("Pioneer (Mackay) — Ωarag vs Dust (Z-scores, −8.8 m, 3-month mean)", y=1.02)
plt.tight_layout()
plt.show()
# Save the figure
fig.savefig("Ocean_3_mackay_omega_vs_dust.png", dpi=600, bbox_inches="tight")

In [15]:
# Same lag analysis for dust
fig, axes = plt.subplots(nrows=len(sites), ncols=1, figsize=(10, 10), sharex=True)
for ax, site in zip(axes, sites):
    sd = (mackay[mackay["Site"] == site]
          .sort_values("Date")
          .set_index("Date")[["omega_z_roll", "dust_z_roll"]]
          .dropna())
    r = pd.Series(
        [sd["omega_z_roll"].corr(sd["dust_z_roll"].shift(k)) for k in LAGS],
        index=list(LAGS))
    ax.plot(r.index, r.values, marker="o", lw=1.8, color="black")
    ax.axhline(0, color="gray", lw=0.7)
    ax.set_ylabel(site)
    k_min = r.idxmin()
    v_min = r.min()
    ax.scatter(k_min, v_min, color="red", zorder=5)
    ax.text(k_min, v_min + 0.08, f"{v_min:.2f}", color="red",
            ha="center", va="bottom", fontsize=16, fontweight="bold")
axes[-1].set_xlabel("Lag (months) — dust leads Ωarag")
fig.suptitle("Pioneer (Mackay) — Lag correlation: Dust leads Ωarag (Z-scores, −8.8 m, 3-month mean)", y=1.02)

plt.tight_layout()
plt.show()
fig.savefig("Ocean_4_mackay_lag_omega_vs_dust_annotated.png", dpi=600, bbox_inches="tight")